In [10]:
import json, csv

data_path = 'data1'

comps1 = json.load(open('%s/comps1.json' % data_path, 'r'))

In [12]:
print(comps1[0])

['Search for specific information or messages', 'Enable seamless and secure data transfer', [['F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F']]]


In [14]:
ref_map1 = {}
error = 0
for d, c in zip(data, comps1):
    refines = sum([1 for r in c[2:][0] if r[0] == 'T']) + sum([1 for r in c[2:][0] if r[1] == 'F']) + sum([1 for r in c[2:][0] if r[2] == 'F']) + sum([1 for r in c[2:][0] if r[3] == 'T'])
    refined_by = sum([1 for r in c[2:][0] if r[0] == 'F']) + sum([1 for r in c[2:][0] if r[1] == 'T']) + sum([1 for r in c[2:][0] if r[2] == 'T']) + sum([1 for r in c[2:][0] if r[3] == 'F'])
    none = sum([1 for r in c[2:][0] if r[0] == 'F']) + sum([1 for r in c[2:][0] if r[1] == 'F']) + sum([1 for r in c[2:][0] if r[2] == 'F']) + sum([1 for r in c[2:][0] if r[3] == 'F'])
    ref_ratio = refines / (refines + refined_by)
    if ref_ratio < 0.3:
        ref_map1['%s:%s' % (d[0], d[1])] = [d[0], d[1], ref_ratio]
    elif ref_ratio > 0.7:
        ref_map1['%s:%s' % (d[0], d[1])] = [d[0], d[1], ref_ratio]
       
print('Found %i/%i probable refinements.' % (len(ref_map1), len(comps1)))

Found 96/550 probable refinements.


In [16]:
data2 = json.load(open('%s/comps2.json' % data_path, 'r'))
print(len(data2))

1099


In [17]:
print(data2[0])

['Search for specific information or messages', 'Enable seamless and secure data transfer', [['A', 'A', 'B', 'A', 'C', 'C'], ['A', 'A', 'A', 'A', 'C', 'C'], ['A', 'A', 'B', 'A', 'C', 'C'], ['A', 'A', 'B', 'A', 'C', 'C'], ['A', 'A', 'B', 'A', 'C', 'C']]]


In [18]:
ref_map2 = {}
error = 0
for d in data2:
    if d[0] == '' or d[1] == '':
        error += 1
        continue
    refines = sum([1 for r in d[2:][0] if r[0] == 'A']) + sum([1 for r in d[2:][0] if r[1] == 'B']) + sum([1 for r in d[2:][0] if r[2] == 'B']) + sum([1 for r in d[2:][0] if r[3] == 'A']) + sum([1 for r in d[2:][0] if r[4] == 'A']) + sum([1 for r in d[2:][0] if r[5] == 'B'])
    refined_by = sum([1 for r in d[2:][0] if r[0] == 'B']) + sum([1 for r in d[2:][0] if r[1] == 'A']) + sum([1 for r in d[2:][0] if r[2] == 'A']) + + sum([1 for r in d[2:][0] if r[3] == 'B']) + sum([1 for r in d[2:][0] if r[4] == 'B']) + sum([1 for r in d[2:][0] if r[5] == 'A'])

    if refines + refined_by == 0:
        continue

    ref_ratio = refines / (refines + refined_by)
    
    if ref_ratio < 0.4 or ref_ratio > 0.6:
        #print('G1: %s\nG2: %s\n[%0.3f] refines: %i / refined_by: %i\n' % (d[0], d[1], ref_ratio, refines, refined_by))
        key = '%s:%s' % (d[0], d[1])
        ref_map2[key] = [d[0], d[1], ref_ratio]
        
print('Found %i/%i probable refinements.' % (len(ref_map2), len(data2) - error))

Found 371/550 probable refinements.


In [20]:
import csv

data = [r for r in csv.reader(open('%s/comps_unlabeled.csv' % data_path, 'r'))]
del data[0]
print(len(data))

550


In [23]:
tp = 0
tn = 0
fp = 0
fn = 0
cc = 0

for r in data:
    key = '%s:%s' % (r[0], r[1])
    rel = 'NONE'
    if key in ref_map1:
        if ref_map1[key][2] < 0.5:
            rel = 'REFINES'
        elif ref_map1[key][2] > 0.5:
            rel = 'REFINED_BY'
    #print('%s : %s' % (r[2], rel))
    if rel == r[2]:
        cc += 1
    
    if r[2] == 'NONE':
        if not key in ref_map1:
            tn += 1
        else:
            fp += 1
    else:
        if key in ref_map1:
            tp += 1
        else:
            fn += 1

print('Accuracy 1: %0.3f' % ((tp + tn) / (tp + tn + fp + fn)))
print('Correctness: %i/%i = %0.3f' % (cc, len(data), cc / len(data)))

Accuracy 1: 0.282
Correctness: 0/550 = 0.000


In [49]:
tp = 0
tn = 0
fp = 0
fn = 0
cc = 0

for r in rows:
    key = '%s:%s' % (r[0], r[1])
    rel = 'NONE'
    if key in ref_map2:
        if ref_map2[key][2] < 0.5:
            rel = 'REFINES'
        elif ref_map2[key][2] > 0.5:
            rel = 'REFINED_BY'
    
    #print('%s : %s' % (r[2], rel))
    if rel == r[2]:
        cc += 1
    
    if r[2] == 'NONE':
        if not key in ref_map2:
            tn += 1
        else:
            fp += 1
    else:
        if key in ref_map2:
            tp += 1
        else:
            fn += 1

print('Accuracy 2: %0.3f' % ((tp + tn) / (tp + tn + fp + fn)))
print('Correctness: %i/%i = %0.3f' % (cc, len(rows), cc / len(rows)))

Accuracy 2: 0.578
Correctness: 108/550 = 0.196


In [50]:
ref_map3 = {}
for k in ref_map1:
    if k in ref_map2:
        ref_map3[k] = (ref_map1[k][2] + ref_map2[k][2]) / 2
    else:
        ref_map3[k] = ref_map1[k][2]

for k in ref_map2:
    if not k in ref_map3:
        ref_map3[k] = ref_map2[k][2]

tp = 0
tn = 0
fp = 0
fn = 0

cc = 0

for r in rows:
    key = '%s:%s' % (r[0], r[1])
    rel = 'NONE'
    if key in ref_map3:
        if ref_map3[key] < 0.5:
            rel = 'REFINES'
        elif ref_map3[key] > 0.5:
            rel = 'REFINED_BY'
    
    #print('%s : %s' % (r[2], rel))
    if rel == r[2]:
        cc += 1
    
    if r[2] == 'NONE':
        if not key in ref_map3:
            tn += 1
        else:
            fp += 1
    else:
        if key in ref_map3:
            tp += 1
        else:
            fn += 1

print('Accuracy 2: %0.3f' % ((tp + tn) / (tp + tn + fp + fn)))
print('Correctness: %i/%i = %0.3f' % (cc, len(rows), cc / len(rows)))

Accuracy 2: 0.600
Correctness: 97/550 = 0.176


In [72]:
import csv

keyed = [r for r in csv.reader(open('comps_keyed.csv', 'r'))]
key_map = {}
for r in keyed:
    s = [r[1], r[2]]
    if r[0] == '0':
        s.append('REFINED_BY')
    elif r[0] == '1':
        s.append('NONE')
    elif r[0] == '3':
        s.append('SYMMETRIC')
    key = '%s:%s' % (r[1], r[2])
    key_map[key] = s

In [73]:
import csv


data = [r for r in csv.reader(open('%s/comps_unlabeled.csv' % data_path, 'r'))]
print(len(data))

missing = 0
for r in data:
    key = '%s:%s' % (r[0], r[1])
    if key in key_map:
        s = r[2].split(',')
        key_map[key].append(s[-1])
    else:
        missing += 1
print('Missing: %i' % missing)

550
Missing: 0


In [74]:
with open('error_analysis.csv', 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(['G1', 'G2', 'model', 'human'])
    writer.writerows(key_map.values())